## Pré: Conexão com Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Etapa 1: Instalação das dependências

In [2]:
!pip install tensorflow-gpu==2.1.0
!pip install tqdm
!pip install kaggle
!pip install keras-rectified-adam
!pip install tensorflow-addons
#!pip install tfa-nightly

     |████████████████████████████████| 421.8MB 35kB/s 
     |████████████████████████████████| 3.8MB 46.3MB/s 
     |████████████████████████████████| 450kB 38.3MB/s 
     |████████████████████████████████| 81kB 12.3MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found exis

  Created wheel for keras-rectified-adam: filename=keras_rectified_adam-0.17.0-cp36-none-any.whl size=14781 sha256=34c14d5cdbe35035e2534fa703e1e60d7a3bdccf271024fee189cf40e3b6b118
  Stored in directory: /root/.cache/pip/wheels/7b/01/27/3a934e1a5644f5b93c720422a6ef97034ea78a21ba71cfb549
Successfully built keras-rectified-adam
     |████████████████████████████████| 1.0MB 2.8MB/s 


### Importação das bibliotecas

In [3]:
import zipfile
import os
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import pandas as pd
import sys
import itertools
from keras_radam import RAdam
import shutil

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

Using TensorFlow backend.


'2.1.0'

### Otimizações tensorflow

In [0]:
#from tensorflow.keras.mixed_precision import experimental as mixed_precision
#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_policy(policy)
tf.config.optimizer.set_jit(True)

### Fazendo o download da base de dados de imagens

In [5]:
%%time
if (os.path.isdir('./ISIC_2019_Test_Input')==False):
  !wget --no-check-certificate \
      https://s3.amazonaws.com/isic-challenge-2019/ISIC_2019_Test_Input.zip \
      -O ./skin_cancer_test.zip

--2020-02-11 11:28:01--  https://s3.amazonaws.com/isic-challenge-2019/ISIC_2019_Test_Input.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.129.253
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.129.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3823247963 (3.6G) [application/zip]
Saving to: ‘./skin_cancer_test.zip’

./skin_cancer_test. 100%[===================>]   3.56G  16.9MB/s    in 4m 10s  

2020-02-11 11:32:12 (14.6 MB/s) - ‘./skin_cancer_test.zip’ saved [3823247963/3823247963]

CPU times: user 1.56 s, sys: 258 ms, total: 1.82 s
Wall time: 4min 13s


## Etapa 2: Pré-processamento

### Configurando os caminhos (paths)

In [0]:
test_dir = "./test"

download_path = './'
derm_ai_path_V1 = '/content/drive/My Drive/Colab Notebooks/Dermatologist_AI'
derm_ai_path = '/content/drive/My Drive/Colab Notebooks/Dermatologist_AI/ISIC_2019'
#ground_truth_file = os.path.join(derm_ai_path, 'ground_truth.csv')

In [0]:
def dcp(download_path, path, file):
  if (os.path.isdir(path)==False):
    dataset_path = os.path.join(download_path,file)
    zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
    zip_object.extractall("./")
    zip_object.close()

### Descompactando a base de dados de Treinamento

### Descompactando a base de dados de Testes

In [8]:
%%time
dcp(download_path, './ISIC_2019_Test_Input', 'skin_cancer_test.zip')

CPU times: user 19.5 s, sys: 3.89 s, total: 23.4 s
Wall time: 1min 9s


In [0]:
!rm ./skin_cancer_test.zip

### Criando geradores de dados (Data Generators)

Redimensionando as imagens

    Grandes arquiteturas treinadas suportam somente alguns tamanhos pré-definidos.

Por exemplo: InceptionResNetV2 (que estamos usando) suporta: (299x299).

In [0]:
data_gen_test = ImageDataGenerator(rescale=1/255., 
                                   fill_mode="constant"
)

In [0]:
!mkdir ./test
!mkdir ./test/test
!mv ./ISIC_2019_Test_Input/* ./test/test

In [12]:
test_generator  = data_gen_test.flow_from_directory(test_dir, target_size=(299,299), batch_size=32, shuffle=False, class_mode=None)

Found 8238 images belonging to 1 classes.


#### Exibição de 3 imagens carregadas

In [13]:
#x,y = test_generator.next()
#for i in range(0,3):
#    image = x[i]
#    label = y[i]
#    print (label)
#    plt.imshow(image)
#    plt.show()

ValueError: ignored

## Etapa 3: Construindo o modelo

### Carregando o modelo pré-treinado (InceptionResNetV2)

### Congelando o modelo base

### Definindo o cabeçalho personalizado da rede neural

### Definindo o modelo

### Compilando o modelo

### Treinando o modelo

### Avaliação do modelo de transferência de aprendizagem

## Etapa 4: Fine tuning


Algumas questões principais:

- NEM SEMPRE é necessário realizar o Fine Tuning em toda a rede neural, pois somente em algumas camadas já é suficiente. A ideia é adotar parte específica da rede neural para nosso problema específico
- Inicie o Fine Tuning DEPOIS que você finalizou a transferência de aprendizagem. Se você tentar o Fine Tuning imediatamente, os gradientes serão muito diferentes entre o cabeçalho personalizado e algumas camadas descongeladas do modelo base 
- Mais informações em como usar o Fine Tuning acesse: 
 https://www.analyticsvidhya.com/blog/2017/06/transfer-learning-the-art-of-fine-tuning-a-pre-trained-model/

### Descongelando algumas camadas do topo do modelo base

Descomentar trecho abaixo para manter camadas congeladas. O número informado no parâmetro `fine_tuning_at` define o número das primeiras camadas que ficarão congeladas. Supondo que a rede tenha 780 camadas e seja informado o valor 500, até a camada 500 a rede ficará congelada.

### Compilando o modelo para fine tuning

### Fine tuning

### Avaliação do modelo com fine tuning

### Salvar modelo após Fine Tuning

In [14]:
%%time
img_shape = (299, 299, 3)
base_model = tf.keras.applications.InceptionResNetV2(input_shape = img_shape, 
                                               include_top = False,
                                               weights = "imagenet")

#base_model.output
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
#global_average_layer
prediction_layer = tf.keras.layers.Dense(units = 8, activation = "softmax")(global_average_layer)

model = tf.keras.models.Model(inputs = base_model.input, outputs = prediction_layer)

radam = tfa.optimizers.RectifiedAdam(
    lr=1e-2,
    total_steps=634,
    warmup_proportion=0.1,
    min_lr=1e-5)
ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)

optm = ranger

model.compile(optimizer=optm, loss="categorical_crossentropy", metrics=[tf.keras.metrics.CategoricalAccuracy()])

219062272/219055592 [==============================] - 14s 0us/step
CPU times: user 8.51 s, sys: 1.07 s, total: 9.58 s
Wall time: 24.4 s


### Carregar modelo após Fine Tuning com os melhores pesos

In [15]:
%%time

#InceptionResNetV2_model_file = os.path.join(derm_ai_path, 'InceptionResNetV2_full_cat.h5')
checkpoint_filepath = os.path.join(derm_ai_path, 'InceptionResNetV2_cat_train2.weights.best.my.hdf5')

# Recreate the exact same model, including its weights and the optimizer
#model = tf.keras.models.load_model(InceptionResNetV2_model_file)
model.load_weights(checkpoint_filepath)


CPU times: user 747 ms, sys: 134 ms, total: 880 ms
Wall time: 5.31 s


### Matriz de Validação

### Execução das predições na base de testes

In [16]:
%%time

test_generator.reset()
pred=model.predict_generator(test_generator,
                              verbose=1)

Instructions for updating:
Please use Model.predict, which supports generators.
258/258 [==============================] - 165s 641ms/step
CPU times: user 3min 2s, sys: 19.1 s, total: 3min 21s
Wall time: 2min 45s


### Identificar classe predita

In [0]:
#def predClasses(pred, train_generator):
#  predicted_class_indices=np.argmax(pred,axis=1)
#
#  labels = (train_generator.class_indices)
#  labels = dict((v,k) for k,v in labels.items())
#  predictions = [labels[k] for k in predicted_class_indices]
#  return predicted_class_indices, labels, predictions

#predicted_class_indices, labels, predictions = predClasses(pred, train_generator)

In [0]:
#image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
#labels[5]
#predicted_class_indices[0]
#predictions[0]
#predicted_class_indices[0]
#pred[0].max()

In [0]:
def is_unknown(pred, th):
  unk = []
  for row in pred:
    if row.max() < th:
        #Unknown class
        unk.append(1.0)
    else:
        unk.append(0.0)
  return unk

th = 0.35

### Salvar resultado para conferência

In [0]:
pd.set_option('display.float_format', lambda x: '%.12f' % x)
results_file = os.path.join(derm_ai_path, 'results_InceptionResNetV2_full_cat_train2.csv')

def saveResultsFile(results_file,pred, test_generator, unk):
  filenames=test_generator.filenames
  filenames = [w.replace('test/', '') for w in filenames]
  filenames = [w.replace('.jpg', '') for w in filenames]
  results=pd.DataFrame({"image":filenames,
                        "MEL":pred[:,4],
                        "NV":pred[:,5],
                        "BCC":pred[:,1],
                        "AK":pred[:,0],
                        "BKL":pred[:,2],
                        "DF":pred[:,3],
                        "VASC":pred[:,7],
                        "SCC":pred[:,6],
                        "UNK": unk
                        })
  results.to_csv(results_file,index=False, float_format='%.12f')

unk = is_unknown(pred, th)
saveResultsFile(results_file, pred, test_generator, unk)

In [21]:
df_comp = pd.read_csv(results_file)
df_comp.head()

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0034321,0.203547328711,0.453984230757,0.038097400218,0.000831945683,0.292736053467,0.006155699492,0.000630238734,0.004017078318,0.000000000000
1,ISIC_0034322,0.002330396790,0.997622311115,0.000000000222,0.000000027114,0.000043403699,0.000003831037,0.000000000263,0.000000027784,0.000000000000
2,ISIC_0034323,0.002058247570,0.000192369625,0.187593728304,0.033276632428,0.007499927189,0.008421585895,0.000002188480,0.760955333710,0.000000000000
3,ISIC_0034324,0.000118745149,0.998529911041,0.000000000063,0.000000000367,0.001349483500,0.000001870541,0.000000044516,0.000000000070,0.000000000000
4,ISIC_0034325,0.000084095140,0.999881148338,0.000000000455,0.000000000127,0.000019618519,0.000015096262,0.000000041436,0.000000000662,0.000000000000


### Rodar avaliação do modelo em relação a base de testes

### Salvar modelo com score obtido

### Carregar melhor modelo 

## Etapa 5: Tensorflow Lite

### Converter modelo para formato tflite

### Salvar arquivo convertido

### Testar modelo convertido (TFLite)
https://medium.com/towards-artificial-intelligence/testing-tensorflow-lite-image-classification-model-e9c0100d8de3

https://colab.research.google.com/github/frogermcs/TFLite-Tester/blob/master/notebooks/Testing_TFLite_model.ipynb#scrollTo=6WEyRJNsR5uL

https://aihub.cloud.google.com/p/products%2F6ca92c9c-6932-45f8-8f08-15a1a6dfafc7